In [2]:
from langchain.prompts import ChatMessagePromptTemplate
promtp = "愿{subject}与你同在"
chat_message_prompt = ChatMessagePromptTemplate.from_template(
    role="Jedi",
    template=promtp
)
chat_message_prompt.format(subject="上帝")

ChatMessage(content='愿上帝与你同在', additional_kwargs={}, response_metadata={}, role='Jedi')

In [9]:
from langchain.prompts import ChatMessagePromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder, ChatPromptTemplate, MessagesPlaceholder

human_prompt = "用{word_connt}字总结我们迄今为止的对话"

human_message_prompt = HumanMessagePromptTemplate.from_template(human_prompt)

chat_prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="conversion"),
    human_message_prompt
])

In [11]:
from langchain_core.messages import AIMessage, HumanMessage
human_message = HumanMessage(content="学习编程最好的方法是什么")

ai_message = AIMessage(content="""
        1. 选择编程语言：决定你想要学习的语言
        2. 从基础开始：熟悉变量、数据类型和控制结构等基本编程概念
        3. 练习、练习、再练习：学习编程最好的方法是通过实践""")

chat_prompt.format_messages(
    conversion=[human_message, ai_message],
    word_connt="20"
)


[HumanMessage(content='学习编程最好的方法是什么', additional_kwargs={}, response_metadata={}),
 AIMessage(content='\n        1. 选择编程语言：决定你想要学习的语言\n        2. 从基础开始：熟悉变量、数据类型和控制结构等基本编程概念\n        3. 练习、练习、再练习：学习编程最好的方法是通过实践', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='用20字总结我们迄今为止的对话', additional_kwargs={}, response_metadata={})]

In [12]:
from langchain_openai import ChatOpenAI
import os
from openai import api_key, base_url
from transformers import Qwen2_5_VLConfig

from langchain_core.output_parsers.string import StrOutputParser
from langchain.schema import output_parser
# llm = ChatOpenAI(model=)
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "sk-C3gsaWJaOsmjrNv1fIobc4zEEsEkD15T98j80Vc8mNuSE420")
# llm = ChatOpenAI(model="o3-mini", api_key=OPENAI_API_KEY, base_url="https://api.deepbricks.ai/v1/")
QWEN_API_KEY = os.getenv("QWEN_API_KEY", "sk-ypjnechwutigvbyglbkguukzsmzkkxfibauydwkbjrypwojd")
model = ChatOpenAI(model="Qwen/Qwen3-30B-A3B-Instruct-2507", api_key=QWEN_API_KEY, base_url="https://api.siliconflow.cn/v1")



output_parser = StrOutputParser()

/home/zpc/anaconda3/envs/langchain/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
chain = chat_prompt | model | output_parser

chain.invoke({"word_connt": "20", "conversion": [human_message, ai_message]})

'学习编程需选语言，打基础，多实践。'

In [14]:
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate
from numpy import full


full_template = """
{introduction}
{example}
{start}
"""

full_prompt = PromptTemplate.from_template(full_template)



In [ ]:
introduction_template = """你正在冒充{person}"""
introduction_prompt = PromptTemplate.from_template(introduction_template)


PromptTemplate(input_variables=['person'], input_types={}, partial_variables={}, template='你正在冒充{person}')

In [17]:
example_template = """
下面是一个交互实例：
Q: {example_q}
A: {example_a}
"""

example_prompt = PromptTemplate.from_template(example_template)


In [18]:
start_template = """
现在正式开始！
Q:{input}
....
"""


start_prompt = PromptTemplate.from_template(start_template)




In [24]:
input_prompt = [
    ("introduction", introduction_prompt),
    ("example", example_prompt),
    ("start", start_prompt)
]

pipline_prompt = PipelinePromptTemplate(
    final_prompt=full_prompt,
    pipeline_prompts=input_prompt
)



pipline_prompt.input_variables

['input', 'person', 'example_q', 'example_a']

In [27]:
pipline_prompt.format(
    person = "masike",
    example_q = "你最喜欢什么车？",
    example_a = "我最喜欢奔驰",
    input = "你最喜欢什么洗衣机"
)
pipline_prompt

PipelinePromptTemplate(input_variables=['input', 'person', 'example_q', 'example_a'], input_types={}, partial_variables={}, final_prompt=PromptTemplate(input_variables=['example', 'introduction', 'start'], input_types={}, partial_variables={}, template='\n{introduction}\n{example}\n{start}\n'), pipeline_prompts=[('introduction', PromptTemplate(input_variables=['person'], input_types={}, partial_variables={}, template='你正在冒充{person}')), ('example', PromptTemplate(input_variables=['example_a', 'example_q'], input_types={}, partial_variables={}, template='\n下面是一个交互实例：\nQ: {example_q}\nA: {example_a}\n')), ('start', PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='\n现在正式开始！\nQ:{input}\n....\n'))])

In [29]:
chain = pipline_prompt | model | output_parser

chain.invoke({"person": "马斯克",
              "example_q": "你最喜欢什么车",
              "example_a": "我最喜欢奔驰",
              "input": "你最喜欢的社交网站是什么？"})




'我最喜欢的是X（原Twitter）。它让我能直接与全球的用户交流，分享想法，讨论科技和未来。毕竟，开放对话是推动进步的关键。'

In [35]:
%time
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache


set_llm_cache(InMemoryCache())



CPU times: user 6 μs, sys: 0 ns, total: 6 μs
Wall time: 14.3 μs


In [38]:
%time
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("请根据下面主题写一篇小红书营销短文:{topic}")
output_parser = StrOutputParser()
chain = prompt | model | output_parser


chain.invoke({"topic":"绿茶"})

CPU times: user 9 μs, sys: 1 μs, total: 10 μs
Wall time: 19.1 μs


'🍃【一口清甜，爱上这杯「绿」意杀」】🌿\n\n姐妹们！今天必须安利我最近的“心头好”——**手作绿茶**！✨\n\n不是便利店那种泡两分钟就发黄的“速成绿”，是真正的、鲜嫩到能掐出水的**明前龙井+茉莉银针**混合茶汤，一口下去，🍃清香直冲天灵盖，连呼吸都变得轻盈了！\n\n✅ 为什么我疯狂回购？\n✔️ 0糖0添加，喝完不胖还养颜✨（每天一杯，气色红润得像开了美颜）\n✔️ 冷泡3小时，绿得像春天的湖水🌊，颜值爆表！拍照直接发朋友圈收获100+点赞\n✔️ 独家秘制配方：绿茶+迷迭香+柠檬片+冰块，清甜微酸，解腻又提神，下午3点犯困？来一杯，瞬间满血复活⚡\n\n💡小tips：早起空腹喝一杯，肠道都感觉被温柔唤醒～尤其是大姨妈前后，喝一喝，整个人都清爽了！\n\n🍵 我还入手了小众国风茶罐，拆开的那一刻，仿佛置身江南茶园，仪式感直接拉满！\n\n📌真心建议：自饮or送人，都非常有格调！闺蜜喝完眼神发亮：“你这茶，是会发光的吧？”\n\n👉 评论区扣【绿】，私信我送你一份【绿茶冷泡秘方】+我的同款茶具图鉴！  \n#手作茶生活 #绿茶控必看 #养生日常 #好物分享 #小红书种草'

In [ ]:
%time
chain.invoke({"topic":"绿茶"})

In [41]:
from langchain.cache import SQLiteCache
set_llm_cache(SQLiteCache())

chain.invoke({"topic":"绿茶"})


'✨喝出仙女感！一杯绿茶，清爽到心尖上～🌿✨  \n\n姐妹们！！最近真的被这杯「绿」彻底征服了\U0001f979  \n不是普通的绿茶，是那种喝一口就仿佛走进了江南烟雨小院的高级感🍃💦  \n\n🍵 我的日常仪式感：  \n早上8点，阳光刚爬上窗台，泡一杯明前龙井，  \n热气袅袅升起，青绿色的茶汤像刚摘下的嫩芽✨  \n轻轻一抿——清甜回甘，带着淡淡的豆香和草木香，  \n瞬间清醒，比咖啡还提神，还不伤胃！  \n\n💡为什么我疯狂种草它？  \n✅ 天然0添加，每一口都干净到发光  \n✅ 抗氧化小能手，对抗熬夜垮脸超有效✨  \n✅ 减脂期救星！低卡又解腻，奶茶戒掉后它就是我的心头好  \n\n👭 和闺蜜打卡模式：  \n周末约在 Café，一人一杯清茶+小点心，  \n聊聊天、拍拍照，发朋友圈连滤镜都省了，天然好气色！📷💛  \n\n📌小贴士：  \n✨ 冲泡水温别太高（80℃左右），保留最鲜嫩的香气  \n✨ 每天1-2杯就够了，喝出好气色不是梦～  \n\n真的，不要小看一杯绿茶的力量💧  \n它不只是饮品，更是一种生活态度——  \n简单、干净、自带滤镜✨  \n\n#绿茶控 #养生日常 #仙女水 #早安仪式感 #轻盈生活 #小红书种草 #办公室必备  \n\n👉 快来评论区告诉我，你最爱的绿茶是哪款？我们一起变美！💚'

In [46]:
##CSV解析器
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

output_parser = CommaSeparatedListOutputParser()
format_instructions = "你的响应应该是csv格式的逗号分隔值的列表, 例如：'内容1','内容2','内容3'"
prompt = PromptTemplate(
    template="{format_instructions}\n 请列出5个{subject}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},

)

chain = prompt | model | output_parser

chain.invoke({"subject": "冰激凌口味"})

["'香草'", "'巧克力'", "'草莓'", "'抹茶'", "'芒果'"]

In [47]:
prompt.invoke({"subject": "冰激凌口味"})

StringPromptValue(text="你的响应应该是csv格式的逗号分隔值的列表, 例如：'内容1','内容2','内容3'\n 请列出5个冰激凌口味")

In [ ]:
## 日期解释器




CPU times: user 9 μs, sys: 1e+03 ns, total: 10 μs
Wall time: 18.4 μs


'🍃【一口清甜，爱上这杯「绿」意杀」】🌿\n\n姐妹们！今天必须安利我最近的“心头好”——**手作绿茶**！✨\n\n不是便利店那种泡两分钟就发黄的“速成绿”，是真正的、鲜嫩到能掐出水的**明前龙井+茉莉银针**混合茶汤，一口下去，🍃清香直冲天灵盖，连呼吸都变得轻盈了！\n\n✅ 为什么我疯狂回购？\n✔️ 0糖0添加，喝完不胖还养颜✨（每天一杯，气色红润得像开了美颜）\n✔️ 冷泡3小时，绿得像春天的湖水🌊，颜值爆表！拍照直接发朋友圈收获100+点赞\n✔️ 独家秘制配方：绿茶+迷迭香+柠檬片+冰块，清甜微酸，解腻又提神，下午3点犯困？来一杯，瞬间满血复活⚡\n\n💡小tips：早起空腹喝一杯，肠道都感觉被温柔唤醒～尤其是大姨妈前后，喝一喝，整个人都清爽了！\n\n🍵 我还入手了小众国风茶罐，拆开的那一刻，仿佛置身江南茶园，仪式感直接拉满！\n\n📌真心建议：自饮or送人，都非常有格调！闺蜜喝完眼神发亮：“你这茶，是会发光的吧？”\n\n👉 评论区扣【绿】，私信我送你一份【绿茶冷泡秘方】+我的同款茶具图鉴！  \n#手作茶生活 #绿茶控必看 #养生日常 #好物分享 #小红书种草'